In [6]:
! export HF_ENDPOINT=https://hf-mirror.com

In [7]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel

agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=InferenceClientModel())

agent.run("Search for the best music recommendations for a party at the Wayne's mansion.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search for the best music recommendations for a party at the Wayne's mansion.                                   │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[Step 1: Duration 0.25 seconds]

KeyboardInterrupt: 

In [ ]:
from smolagents import CodeAgent, tool, InferenceClientModel

# 根据场合建议菜单的工具
@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion: The type of occasion for the party.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

# 管家阿尔弗雷德，为派对准备菜单
agent = CodeAgent(tools=[suggest_menu], model=InferenceClientModel())

# 为派对准备正式菜单
agent.run("Prepare a formal menu for the party.")

In [9]:
from smolagents import CodeAgent, tool,InferenceClientModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 1️⃣ 定义工具函数
@tool
def catering_service_tool(query: str) -> str:
    """
    Returns the highest-rated catering service in Gotham City based on a query.

    Args:
        query: A search term to filter catering services.

    Returns:
        The name of the best-rated catering service.
    """
    services = {
        "Gotham Catering Co.": 4.9,
        "Wayne Manor Catering": 4.8,
        "Gotham City Events": 4.7,
    }
    best_service = max(services, key=services.get)
    return best_service

# 2️⃣ 加载 Qwen 模型
model_name = "/root/All_in_llm/models/Qwen/Qwen1.5-0.5B-Chat"  # 可改为本地路径
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=True).eval()

# 3️⃣ 自定义一个简单的模型适配器（供 SmolAgents 使用）
class QwenAgentModel:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def generate(self, prompt: str) -> str:
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        with torch.no_grad():
            output = self.model.generate(**inputs, max_new_tokens=200)
        return self.tokenizer.decode(output[0], skip_special_tokens=True)

qwen_model = QwenAgentModel(model, tokenizer)

# 4️⃣ 创建 CodeAgent
agent = CodeAgent(tools=[catering_service_tool], model = InferenceClientModel(qwen_model))

# 5️⃣ 运行智能体
result = agent.run("Can you give me the name of the highest-rated catering service in Gotham City?")
print(result)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give me the name of the highest-rated catering service in Gotham City?                                  │
│                                                                                                                 │
╰─ InferenceClientModel - <__main__.QwenAgentModel object at 0x7fd2438a0a40> ─────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
'QwenAgentModel' object has no attribute 'split'

[Step 1: Duration 0.00 seconds]

AgentGenerationError: Error in generating model output:
'QwenAgentModel' object has no attribute 'split'